<a href="https://colab.research.google.com/github/vdat-18/ngvdat/blob/main/scrape_tiki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***`________________________________________________________________________________________________________________________________________________________________`***

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install requests

In [ ]:
import requests
import pandas
import time
import random
import pandas as pd
from tqdm import tqdm
from openpyxl import load_workbook
import json
from multiprocessing import Pool, cpu_count
import re
from tqdm import tqdm

# ***`GET "PRODUCT_ID"`***

In [ ]:
import requests

headers = {
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'en-US,en;q=0.8',
    '^cookie': 'TOKENS=^{^%^22access_token^%^22:^%^22sGXVTeHDn9da0MgmvSpQcN5PLWlrAZ7C^%^22^}; _trackity=9e8a0644-a99d-86b4-83f1-868b4f808281; TIKI_RECOMMENDATION=f78133de4856b4d837775f81483ed4a9; TKSESSID=7b6645586fb80f08e6560923e3f04f37; tiki_client_id=; delivery_zone=Vk4wMzkwMjMwMTE=^',
    'priority': 'u=1, i',
    'referer': 'https://tiki.vn/tui-thoi-trang-nam/c27616',
    '^sec-ch-ua': '^\\^Not/A)Brand^\\^;v=^\\^8^\\^, ^\\^Chromium^\\^;v=^\\^126^\\^, ^\\^Brave^\\^;v=^\\^126^\\^^',
    'sec-ch-ua-mobile': '?0',
    '^sec-ch-ua-platform': '^\\^Windows^\\^^',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'sec-gpc': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36',
    'x-guest-token': 'sGXVTeHDn9da0MgmvSpQcN5PLWlrAZ7C',
}

params = (
    ('limit', '40'),
    ('include', 'advertisement'),
    ('aggregations', '2'),
    ('version', 'home-persionalized'),
    ('trackity_id', '9e8a0644-a99d-86b4-83f1-868b4f808281'),
    ('category', '27616'),
    ('page', '1'),
    ('urlKey', 'tui-thoi-trang-nam'),
)

response = requests.get('https://tiki.vn/api/personalish/v1/blocks/listings', headers=headers, params=params)

#NB. Original query string below. It seems impossible to parse and
#reproduce query strings 100% accurately so the one below is given
#in case the reproduced version is not "correct".
# response = requests.get('https://tiki.vn/api/personalish/v1/blocks/listings?limit=40&include=advertisement&aggregations=2&version=home-persionalized&trackity_id=9e8a0644-a99d-86b4-83f1-868b4f808281&category=27616&page=1&urlKey=tui-thoi-trang-nam', headers=headers)


In [ ]:
params = dict(params)

In [ ]:
product_id = []
for i in range (1,6):
  params['page'] = i
  response = requests.get('https://tiki.vn/api/personalish/v1/blocks/listings', headers = headers, params = params)
  if response.status_code == 200:
    print("request success!!!")
    for record in response.json().get('data'):
      product_id.append({'id': record.get('id')})
      # product_id.append({'brand_name': record.get('brand_name')})

  time.sleep(random.randrange(3,10))
df = pd.DataFrame(product_id)

In [ ]:
df.drop_duplicates(inplace=True)

# ***`________________________________________________________________________________________________________________________________________________________________`***

# ***`GET "NAME_SHOP" and "LINK_SHOP"`***

In [ ]:
def parser_product(json):
    d = dict()
    # Handle the case when 'current_seller' might be missing
    current_seller = json.get('current_seller')
    if current_seller:
        d['id'] = current_seller.get('id')
        d['link'] = current_seller.get('link')
        d['name'] = current_seller.get('name')
    else:
        # Handle the case gracefully, e.g., set default values or log a warning
        d['id'] = None
        d['link'] = None
        d['name'] = None
        print(f"Warning: 'current_seller' not found in JSON for product ID {json.get('id', 'Unknown')}")
    return d

In [ ]:
# Assuming 'df' is your DataFrame with product IDs
df_id = df
p_ids = df_id.id.to_list()


def fetch_product_data(pid):
    response = requests.get(f'https://tiki.vn/api/v2/products/{pid}', headers=headers, params=params)
    if response.status_code == 200:
        try:
            return parser_product(response.json())
        except ValueError as e:
            print(f"Error decoding JSON for product ID {pid}: {e}")
            return None
    else:
        print(f"Request failed for product ID {pid} with status code: {response.status_code}")
        return None

def worker(p_ids_chunk):
    result_chunk = []
    for pid in p_ids_chunk:
        product_data = fetch_product_data(pid)
        if product_data is not None:
            result_chunk.append(product_data)
        time.sleep(random.randrange(3, 5))  # Adjust sleep time if needed
    return result_chunk

if __name__ == '__main__':
    # Determine the number of worker processes
    num_workers = multiprocessing.cpu_count()

    # Split the product IDs into chunks
    chunk_size = len(p_ids) // num_workers
    chunks = [p_ids[i:i + chunk_size] for i in range(0, len(p_ids), chunk_size)]

    # Create a pool of worker processes
    pool = multiprocessing.Pool(num_workers)

    # Use tqdm to show progress bar
    result = []
    for chunk_result in tqdm(pool.imap(worker, chunks), total=len(chunks)):
        result.extend(chunk_result)

    # Close the pool and wait for the worker processes to finish
    pool.close()
    pool.join()

    # Create DataFrame from the results
    df_product = pd.DataFrame(result)


100%|██████████| 2/2 [06:53<00:00, 206.72s/it]


In [ ]:
df_product.drop_duplicates(inplace=True)

# ***`________________________________________________________________________________________________________________________________________________________________`***

# ***`GET "PRODUCT_ID" OF EACH PRODUCT IN A SHOP`***

In [ ]:
import requests

headers = {
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'en-US,en;q=0.8',
    '^cookie': '_trackity=bbb05fd0-d2f9-29d5-7332-31137c25fb41; TOKENS=^{^%^22access_token^%^22:^%^22sGXVTeHDn9da0MgmvSpQcN5PLWlrAZ7C^%^22^}; delivery_zone=Vk4wMzkwMDYwMDE=; tiki_client_id=^',
    'origin': 'https://tiki.vn',
    'priority': 'u=1, i',
    'referer': 'https://tiki.vn/',
    '^sec-ch-ua': '^\\^Not)A;Brand^\\^;v=^\\^99^\\^, ^\\^Brave^\\^;v=^\\^127^\\^, ^\\^Chromium^\\^;v=^\\^127^\\^^',
    'sec-ch-ua-mobile': '?0',
    '^sec-ch-ua-platform': '^\\^Windows^\\^^',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'sec-gpc': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36',
    'x-guest-token': 'sGXVTeHDn9da0MgmvSpQcN5PLWlrAZ7C',
}

params = (
    ('limit', '1'),
    ('page', '1'),
)

response = requests.get('https://api.tiki.vn/v2/seller/stores/shop-mrhi/products', headers=headers, params=params)

#NB. Original query string below. It seems impossible to parse and
#reproduce query strings 100% accurately so the one below is given
#in case the reproduced version is not "correct".
# response = requests.get('https://api.tiki.vn/v2/seller/stores/shop-mrhi/products?limit=1&page=1', headers=headers)


In [ ]:
params = dict(params)
product_id_eachProduct = []
for i in range(1, 4):
    params['page'] = i
    response = requests.get('https://api.tiki.vn/v2/seller/stores/shop-mrhi/products', headers=headers, params=params)

    if response.status_code == 200:
        for record in response.json().get('data'):
            product_id_eachProduct.append({'id': record.get('id')})
df = pd.DataFrame(product_id_eachProduct)

# ***`GET INFOMATION OF EACH PRODUCT REGARDING "PRODUCT_ID"`***

In [ ]:
import requests

headers = {
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'en-US,en;q=0.8',
    '^cookie': '_trackity=bbb05fd0-d2f9-29d5-7332-31137c25fb41; TOKENS=^{^%^22access_token^%^22:^%^22sGXVTeHDn9da0MgmvSpQcN5PLWlrAZ7C^%^22^}; delivery_zone=Vk4wMzkwMDYwMDE=; tiki_client_id=^',
    'priority': 'u=1, i',
    'referer': 'https://tiki.vn/cua-hang/shop-mrhi?t=product',
    '^sec-ch-ua': '^\\^Not)A;Brand^\\^;v=^\\^99^\\^, ^\\^Brave^\\^;v=^\\^127^\\^, ^\\^Chromium^\\^;v=^\\^127^\\^^',
    'sec-ch-ua-mobile': '?0',
    '^sec-ch-ua-platform': '^\\^Windows^\\^^',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'sec-gpc': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36',
    'x-guest-token': 'sGXVTeHDn9da0MgmvSpQcN5PLWlrAZ7C',
}

params = (
    ('platform', 'web'),
    ('spid', '144643834'),
    ('version', '3'),
)

response = requests.get('https://tiki.vn/api/v2/products/144643830', headers=headers, params=params)

#NB. Original query string below. It seems impossible to parse and
#reproduce query strings 100% accurately so the one below is given
#in case the reproduced version is not "correct".
# response = requests.get('https://tiki.vn/api/v2/products/144643830?platform=web&spid=144643834&version=3', headers=headers)


In [ ]:
def parser_product(json):
    d = dict()
    d['id'] = json.get('id')
    d['name'] = json.get('name')
    d['price'] = json.get('price')
    d['rating_average'] = json.get('rating_average')
    d['review_count'] = json.get('review_count')
    d['quantity_sold'] = json.get('all_time_quantity_sold')
    d['categories'] = json.get('categories').get('name')

    return d
#-------------------------------------------------------------------------------
df_id = df
p_ids = df_id.id.to_list()
result = []
for pid in tqdm(p_ids, total=len(p_ids)):
    response = requests.get('https://tiki.vn/api/v2/products/{}'.format(pid), headers=headers, params=params)

    # Check for non-200 status codes and handle them
    if response.status_code != 200:
        print(f"Error fetching data for product ID: {pid}, Status Code: {response.status_code}")
        continue  # Skip to the next product ID

    try:
        result.append(parser_product(response.json()))
    except ValueError as e:
        print(f"Error decoding JSON for product ID: {pid}, Error: {e}")
        # print(response.text)  # Print the raw response for debugging

df_product_mapping = pd.DataFrame(result)

100%|██████████| 120/120 [00:57<00:00,  2.08it/s]

Error decoding JSON for product ID: 73167033, Error: Expecting value: line 1 column 1 (char 0)


# ***`GET "REVIEWS" OF EACH PRODUCT`***

In [ ]:
import requests

headers = {
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'en-US,en;q=0.8',
    '^cookie': '_trackity=bbb05fd0-d2f9-29d5-7332-31137c25fb41; TOKENS=^{^%^22access_token^%^22:^%^22sGXVTeHDn9da0MgmvSpQcN5PLWlrAZ7C^%^22^}; delivery_zone=Vk4wMzkwMDYwMDE=; tiki_client_id=^',
    'priority': 'u=1, i',
    'referer': 'https://tiki.vn/op-lung-kinh-cuong-luc-nham-chong-van-tay-baseus-frosted-glass-protective-case-dung-cho-iphone-13-series_-hang-chinh-hang-p144643830.html?spid=144643834',
    '^sec-ch-ua': '^\\^Not)A;Brand^\\^;v=^\\^99^\\^, ^\\^Brave^\\^;v=^\\^127^\\^, ^\\^Chromium^\\^;v=^\\^127^\\^^',
    'sec-ch-ua-mobile': '?0',
    '^sec-ch-ua-platform': '^\\^Windows^\\^^',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'sec-gpc': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36',
    'x-guest-token': 'sGXVTeHDn9da0MgmvSpQcN5PLWlrAZ7C',
}

params = (
    ('limit', '5'),
    ('include', 'comments,contribute_info,attribute_vote_summary'),
    ('sort', 'score^%^7Cdesc,id^%^7Cdesc,stars^%^7Call'),
    ('page', '1'),
    ('spid', '144643834'),
    ('product_id', '144643830'),
    ('seller_id', '5822^'),
)

response = requests.get('https://tiki.vn/api/v2/reviews', headers=headers, params=params)

#NB. Original query string below. It seems impossible to parse and
#reproduce query strings 100% accurately so the one below is given
#in case the reproduced version is not "correct".
# response = requests.get('^https://tiki.vn/api/v2/reviews?limit=5&include=comments,contribute_info,attribute_vote_summary&sort=score^%^7Cdesc,id^%^7Cdesc,stars^%^7Call&page=1&spid=144643834&product_id=144643830&seller_id=5822^', headers=headers)


In [ ]:
#----------------------------------------------------------------------------------------------------------------------------------------------------------------------
def comment_parser(json):
    d = dict()
    d['content'] = json.get('content')
    return d
#-----------------------------------------------------------------------------------------------------------------------------------------------------------------------
# Load product IDs from Excel file
df_id = df_product_mapping
p_ids = df_id.id.to_list()

# Main list to store results
result = []

# Iterate over product IDs
for pid in tqdm(p_ids, total=len(p_ids)):
    params_dict = dict(params) # Convert tuple to dictionary
    params_dict['product_id'] = pid
    temp_result = []

    # Iterate over pages
    for i in range(1, 21):
        params_dict['page'] = i
        response = requests.get('https://tiki.vn/api/v2/reviews', headers=headers, params=params_dict)
        if response.status_code == 200:
            for comment in response.json().get('data'):
                temp_result.append(comment_parser(comment))

    # Append temporary list to main result list
    result.append(temp_result)
df_comment_mapping = pd.DataFrame(result)


100%|██████████| 119/119 [14:53<00:00,  7.51s/it]


In [ ]:
# Assuming df_product_mapping is your DataFrame
# Replace illegal characters with a suitable replacement
# df_product_mapping = df_product_mapping.replace('\b', '', regex=True)

df_product_mapping.to_excel('/content/drive/MyDrive/Colab Notebooks/Dataset TIki/new_info_1.xlsx', index=False)
df_comment_mapping.to_excel('/content/drive/MyDrive/Colab Notebooks/Dataset TIki/comments_tiki_1.xlsx', index=False)


# ***`________________________________________________________________________________________________________________________________________________________________`***

# ***`Estimated Revenue Crawl`***

Đoạn code dưới đây nhằm thu thập dữ liệu "id", "quantity_sold" sau 1 tháng nhằm tính toán doanh thu từng sản phẩm và ước tính doanh thu cửa hàng.

In [ ]:
df = pd.read_excel('/content/drive/My Drive/Colab Notebooks/NCKH (Ph. Danh Hường)/Data thô/raw_final.xlsx')

In [ ]:
import requests

headers = {
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'en-US,en;q=0.7',
    '^cookie': 'TOKENS=^{^%^22access_token^%^22:^%^22sGXVTeHDn9da0MgmvSpQcN5PLWlrAZ7C^%^22^}; _trackity=76e86747-ffbd-a408-d1c4-465a4492ad49; tiki_client_id=; delivery_zone=Vk4wMzkwMjMwMTE=^',
    'priority': 'u=1, i',
    'referer': 'https://tiki.vn/tai-nghe-nhet-tai-co-micro-jbl-c200si-hang-chinh-hang-p214168962.html?spid=214168963',
    '^sec-ch-ua': '^\\^Not)A;Brand^\\^;v=^\\^99^\\^, ^\\^Brave^\\^;v=^\\^127^\\^, ^\\^Chromium^\\^;v=^\\^127^\\^^',
    'sec-ch-ua-mobile': '?0',
    '^sec-ch-ua-platform': '^\\^Windows^\\^^',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'sec-gpc': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36',
    'x-guest-token': 'sGXVTeHDn9da0MgmvSpQcN5PLWlrAZ7C',
}

params = (
    ('platform', 'web'),
    ('spid', '214168963'),
    ('version', '3'),
)

response = requests.get('https://tiki.vn/api/v2/products/214168962', headers=headers, params=params)

#NB. Original query string below. It seems impossible to parse and
#reproduce query strings 100% accurately so the one below is given
#in case the reproduced version is not "correct".
# response = requests.get('https://tiki.vn/api/v2/products/214168962?platform=web&spid=214168963&version=3', headers=headers)


In [ ]:
def parser_product(json):
    d = dict()
    d['id'] = json.get('id')
    d['quantity_sold'] = json.get('all_time_quantity_sold')

    return d

# ***`Crawling data `***

In [ ]:
df_id = df.iloc[:]
print(df_id)
p_ids = df_id.id.to_list()
p_ids = [int(x) for x in p_ids]
p_ids[:5]

      Source.Name         id          id2  quantity_sold_new  \
20000  File2.xlsx  179562656  179562656.0                2.0   
20001  File2.xlsx  178316505  178316505.0                NaN   
20002  File2.xlsx   93068708   93068708.0                9.0   
20003  File2.xlsx   93009788   93009788.0                NaN   
20004  File2.xlsx  103753899  103753899.0                1.0   
...           ...        ...          ...                ...   
21099  File2.xlsx  270791815  270791815.0                6.0   
21100  File2.xlsx  216401387  216401387.0                5.0   
21101  File2.xlsx  270791335  270791335.0                9.0   
21102  File2.xlsx  272224450  272224450.0                1.0   
21103  File2.xlsx  186627908  186627908.0             2240.0   

       quantity_sold_old                                               name  \
20000                2.0  Thanh lý giày lười nam DONAVY cao cấp đế cao 3...   
20001                NaN  Dép Quai Ngang Nam Da DONAVY Kiểu Quai Con Ngự.

[179562656, 178316505, 93068708, 93009788, 103753899]

# ***`Fill in blank data`***

In [ ]:
# Filter variance which has "id2" is
df_id = df.iloc[15000:15000]
df_null_id2 = df_id[df_id["id2"].isnull()]

# If there are any rows where 'id2' is null
if not df_null_id2.empty:
    # Convert the "id" column of these rows to a list of integers
    p_ids = df_null_id2["id"].tolist()
    # Use a loop to check and handle NaN values
    new_p_ids = []
    for x in p_ids:
        if pd.notna(x): # Check if x is not NaN
            new_p_ids.append(int(x))
    p_ids = new_p_ids

    # Print the first 5 elements (or fewer if there are less than 5)
    print(p_ids[:5])

[32725930]


In [ ]:
result = []
for pid in tqdm(p_ids, total=len(p_ids)):
    response = requests.get('https://tiki.vn/api/v2/products/{}'.format(pid), headers=headers, params=params)
    if response.status_code == 200:
        try:
            result.append(parser_product(response.json()))
        except ValueError as e:  # Handle potential JSONDecodeError
            print(f"Error decoding JSON for product ID {pid}: {e}")
    else:
        print(f"Request failed for product ID {pid} with status code: {response.status_code}")

df_product = pd.DataFrame(result)
df_product.to_excel('/content/drive/MyDrive/Colab Notebooks/nhap1.xlsx')

 33%|███▎      | 1/3 [00:00<00:00,  4.01it/s]

Request failed for product ID 274192275 with status code: 404


 67%|██████▋   | 2/3 [00:00<00:00,  2.53it/s]

Request failed for product ID 274180359 with status code: 404


100%|██████████| 3/3 [00:00<00:00,  3.02it/s]

Request failed for product ID 1066164 with status code: 404


# ***`Merge `***

In [ ]:
df_id = df.iloc[60000:65000]
print(df_id.head(5))
df_fill_blank = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/nhap1.xlsx')
print(df_fill_blank.head(5))

      Source.Name        id  id2  quantity_sold_new  quantity_sold_old  \
60000  File5.xlsx   2683363  NaN                NaN               44.0   
60001  File5.xlsx  44573182  NaN                NaN                4.0   
60002  File5.xlsx   2507979  NaN                NaN               10.0   
60003  File5.xlsx   1994753  NaN                NaN                2.0   
60004  File5.xlsx   2000679  NaN                NaN                4.0   

                                                    name    price  \
60000        Giày Boot Nữ Cột Dây Huy Hoàng HT7920 - Đen   999000   
60001                    Giày nữ Huy Hoàng màu đỏ HT7065   489000   
60002   Dây Đồng Hồ Da Cá Sấu Huy Hoàng HT8202 - Nâu Đất   279000   
60003  Thắt Lưng Nam Da Cá Sấu Gai Bụng Huy Hoàng HT4...  1099000   
60004  Ví Nam Da Trăn Nguyên Thủy Kiểu Đứng Huy Hoàng...   599000   

       rating_average  review_count               categories  ...   85   86  \
60000             4.0             1               Thời Trang 

In [ ]:
# Iterate through df_fill_blank and update df_id
for index, row in df_fill_blank.iterrows():
    mask = df_id['id'] == row['id']
    df_id.loc[mask, 'id2'] = row['id']
    df_id.loc[mask, 'quantity_sold_new'] = row['quantity_sold']

# Display the updated df_id
print(df_id)

      Source.Name         id          id2  quantity_sold_new  \
60000  File5.xlsx    2683363    2683363.0               44.0   
60001  File5.xlsx   44573182   44573182.0                4.0   
60002  File5.xlsx    2507979    2507979.0               10.0   
60003  File5.xlsx    1994753    1994753.0                7.0   
60004  File5.xlsx    2000679    2000679.0                NaN   
...           ...        ...          ...                ...   
63761  File5.xlsx   15271684   15271684.0                NaN   
63762  File5.xlsx  192447521  192447521.0                NaN   
63763  File5.xlsx  202879719  202879719.0                2.0   
63764  File5.xlsx  208073919  208073919.0                NaN   
63765  File5.xlsx  189474023  189474023.0              524.0   

       quantity_sold_old                                               name  \
60000               44.0        Giày Boot Nữ Cột Dây Huy Hoàng HT7920 - Đen   
60001                4.0                    Giày nữ Huy Hoàng màu đỏ HT70

In [ ]:
df_new = df_id.iloc[:, :13]
print(df_new)

      Source.Name         id          id2  quantity_sold_new  \
60000  File5.xlsx    2683363    2683363.0               44.0   
60001  File5.xlsx   44573182   44573182.0                4.0   
60002  File5.xlsx    2507979    2507979.0               10.0   
60003  File5.xlsx    1994753    1994753.0                7.0   
60004  File5.xlsx    2000679    2000679.0                NaN   
...           ...        ...          ...                ...   
63761  File5.xlsx   15271684   15271684.0                NaN   
63762  File5.xlsx  192447521  192447521.0                NaN   
63763  File5.xlsx  202879719  202879719.0                2.0   
63764  File5.xlsx  208073919  208073919.0                NaN   
63765  File5.xlsx  189474023  189474023.0              524.0   

       quantity_sold_old                                               name  \
60000               44.0        Giày Boot Nữ Cột Dây Huy Hoàng HT7920 - Đen   
60001                4.0                    Giày nữ Huy Hoàng màu đỏ HT70

In [ ]:
df_new.to_excel('/content/drive/MyDrive/Colab Notebooks/nhap10.xlsx')

# ***`Basic Preprocessing`***

# ***`Remove White Space (df_combined_all)`***

In [ ]:
df_combined_all = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Dataset TIki/df_combined_all.xlsx')

In [ ]:
df_combined_all.head(5)

,id,link,name,Unnamed: 3,Unnamed: 4,Date,Unnamed: 6,Year_Joined,Shop_Rating,Counter_Rating,Followers,Chat Resopnse
0,38963,https://tiki.vn/cua-hang/techmorevn,Techmorevn,NaN,done,18/07/2024,NaN,2019,4.7,1600,693,91
1,235971,https://tiki.vn/cua-hang/vivo-official-store,vivo Official Store,NaN,done,NaN,NaN,2021,4.6,44,443,0
2,113592,https://tiki.vn/cua-hang/vien-thong-ha-nhung,Viễn thông Hà Nhung,NaN,done,NaN,NaN,2022,4.9,390,114,86
3,106499,https://tiki.vn/cua-hang/shop-may-doc-sach-ha-noi,Shop Máy Đọc Sách Hà Nội,NaN,done,NaN,NaN,2020,4.8,877,499,0
4,29952,https://tiki.vn/cua-hang/hong-hanh-mobile,Hồng Hạnh Mobile,file 1,done,NaN,NaN,2019,4.7,2800,15108,100


# ***Chat Response***

In [ ]:
print(df_combined_all['Chat Resopnse'].unique())

[91 0 86 100 94 61 nan 80 88 96 93 57 92 95 74 90 83 60 'Chưa' 50 66 82 53
 72 75 71 85 70 87 62 78 73 77 98 89 69 76 99 84 81 54 0.85]


In [ ]:
df_combined_all['Chat Resopnse'] = df_combined_all['Chat Resopnse'].replace('Chưa', 0)
df_combined_all['Chat Resopnse'] = df_combined_all['Chat Resopnse'].replace(0.85, 85)

In [ ]:
print(df_combined_all['Chat Resopnse'].unique())

[ 91.   0.  86. 100.  94.  61.  nan  80.  88.  96.  93.  57.  92.  95.
  74.  90.  83.  60.  50.  66.  82.  53.  72.  75.  71.  85.  70.  87.
  62.  78.  73.  77.  98.  89.  69.  76.  99.  84.  81.  54.]


In [ ]:
print(df_combined_all['Chat Resopnse'].unique())

[ 91.   0.  86. 100.  94.  61.  nan  80.  88.  96.  93.  57.  92.  95.
  74.  90.  83.  60.  50.  66.  82.  53.  72.  75.  71.  85.  70.  87.
  62.  78.  73.  77.  98.  89.  69.  76.  99.  84.  81.  54.]


In [ ]:
for i in df_combined_all['Chat Resopnse']:
  if type(i) == str:
    print(i)

## **Followers**

In [ ]:
print(df_combined_all['Followers'].unique())

[693 443 114 499 15108 148 1483 2199 5435 nan 31 178 224 234 228 90 8 559
 80 891 249 38 10 21 970 166 2072 18 2022 3043 8964 1057 565 14639 1709
 410 '2.2k' 246 1891 376 476 206 520 590 2503 29 16 3673 1717 765 536 92
 3517 655 652 325 1175 902 1478 3962 174 1889 4830 20 537 923 128 1576
 1005 1207 2068 3602 60 820 75 532 296 78 13324 493 4206 136 362 6448 4636
 279 2963 742 85 394 12798 957 601 243 101 2560 1019 147 12571 2583 835
 1519 2684 175 18106 454 '3.4k' 1484 1496 761 2524 3719 2210 535 8780 5587
 218 457 260 941 1654 469 834 2781 336 126 516 102 145 971 2 239 883 1500
 1584 144 1051 252 298 129 801 3254 308 4901 132 352 103 3726 3105 2156
 1606 3245 125 1550 33 613 1699 240 5275 45 429 36 3 1339 803 253 143 3318
 1375 98 5701 343 522 711 12 3741 444 697 702 370 19 4280 628 551 5369 54
 1645 464 58 2291 737 1899 12254 6333 250 3537 2611 1004 682 2836 1368
 2925 2510 947 391 2530 816 11184 2814 2491 3013 211 1287 337 140 1439 354
 283 1863 1567 397 965 194 277 14 97 611 479 98

In [ ]:
df_combined_all['Followers'] = df_combined_all['Followers'].replace('2.2k', 2200)
df_combined_all['Followers'] = df_combined_all['Followers'].replace('3.4k', 3400)
df_combined_all['Followers'] = df_combined_all['Followers'].replace('8.2k', 8200)
df_combined_all['Followers'] = df_combined_all['Followers'].replace('1.1k', 1100)
df_combined_all['Followers'] = df_combined_all['Followers'].replace('1.3k', 1300)
df_combined_all['Followers'] = df_combined_all['Followers'].replace('1.5k', 1500)
df_combined_all['Followers'] = df_combined_all['Followers'].replace('1.7k', 1700)
df_combined_all['Followers'] = df_combined_all['Followers'].replace('6.6k', 6600)
df_combined_all['Followers'] = df_combined_all['Followers'].replace('3.8k', 3800)

In [ ]:
for i in df_combined_all['Followers']:
  if type(i) == str:
    print(i)

# ***Counter_Rating***

In [ ]:
print(df_combined_all['Counter_Rating'].unique())

['1.6k+' 44 390 877 '2.8k+' 167 '7.6k' '2.3k' '2.6k' nan 218 '1.3k' 199
 132 '1.1k' 684 124 '5.7k' 134 185 69 12 6 127 4 41 '5.0k' '8.3k' '21k'
 '2.1k' '3.1k' '8.1k' 90 '1.6k' '5.4k' 97 705 576 '3.5k' '1.9k' '1.7k' 356
 378 39 '2.0k' '10k' '3.8k' 629 '17k' '1.5k' '1.2k' 648 '2.7k' '6.4k'
 '1.4k' 798 '14k' 182 220 '11k' 89 274 '6.5k' 327 579 '4.1k' 231 '6.1k'
 164 '23k' 478 '9.9k' 778 '3.4k' '20k' '3.2k' 580 '5.2k' 923 887 '22k'
 '13k' '1.0k' '4.3k' '28k' 737 '5.3k' '2.4k' '9.0k' '5.8k' 751 '7.9k'
 '2.8k' '4.6k' '2.5k' 647 974 228 8 5 '7.3k' 519 '4.2k' '7.0k' '2.2k'
 '15k' 992 '4.9k' '1.8k' 100 296 56 '3.0k' 57 11 '12k' '2.9k' 248 621 896
 '5.6k' 411 '4.4k' 19 '3.7k' '8.6k' 559 291 '19k' '4.0k' 289 229 '9.5k'
 727 651 604 244 375 159 740 805 583 144 782 682 80 245 679 916 819 206
 '8.4k' 261 349 '9.3k' 43 894 7 733 23 '3.3k' '6.2k' 437 '9.8k' 763 '4.5k'
 '6.0k' 373 315 310 384 765 286 352 '24k' 95 140 '5.1k' 211 343 769 634
 924 '8.5k' 588 741 277 451 646 543 226 515 340 555 708 '33k' 8

In [ ]:
def convert_to_thousands(value):
    if isinstance(value, str):  # Check if the value is a string before applying lower()
        value = value.lower()
        if 'k+' in value:
            value = value.replace('k+', '')
        elif 'k' in value:
            value = value.replace('k', '')
        else:
            return int(value)  # Return the integer value if no 'k' is present

        return int(float(value) * 1000)  # Convert to thousands if 'k' was present
    else:
        return value  # Return the value as is if it's not a string

In [ ]:
df_combined_all['Counter_Rating'] = df_combined_all['Counter_Rating'].apply(convert_to_thousands)

In [ ]:
for i in df_combined_all['Counter_Rating']:
  if type(i) == str:
    print(i)

# ***`Shop Rating`***

In [ ]:
print(df_combined_all['Shop_Rating'].unique())

[4.7 4.6 4.9 4.8 4.4 nan 4.5 4.2 5.  4.1 4.3 4.  3.8 3.6 3.9 2.5 3.5 3.7
 3.4 3. ]


# ***`Year_Joined`***

In [ ]:
print(df_combined_all['Year_Joined'].unique())

[2019. 2021. 2022. 2020. 2018.   nan 2017. 2024. 2023.]


# ***`AsType(Int)`***

In [ ]:
# Handle missing or infinite values before converting to integer
df_combined_all['Year_Joined'] = df_combined_all['Year_Joined'].fillna(0).astype(int)
df_combined_all['Counter_Rating'] = df_combined_all['Counter_Rating'].fillna(0).astype(int)
df_combined_all['Followers'] = df_combined_all['Followers'].fillna(0).astype(int)
df_combined_all['Chat Resopnse'] = df_combined_all['Chat Resopnse'].fillna(0).astype(int)


In [ ]:
df_combined_all

,id,link,name,Unnamed: 3,Unnamed: 4,Date,Unnamed: 6,Year_Joined,Shop_Rating,Counter_Rating,Followers,Chat Resopnse
0,38963,https://tiki.vn/cua-hang/techmorevn,Techmorevn,NaN,done,18/07/2024,NaN,2019,4.7,1600,693,91
1,235971,https://tiki.vn/cua-hang/vivo-official-store,vivo Official Store,NaN,done,NaN,NaN,2021,4.6,44,443,0
2,113592,https://tiki.vn/cua-hang/vien-thong-ha-nhung,Viễn thông Hà Nhung,NaN,done,NaN,NaN,2022,4.9,390,114,86
3,106499,https://tiki.vn/cua-hang/shop-may-doc-sach-ha-noi,Shop Máy Đọc Sách Hà Nội,NaN,done,NaN,NaN,2020,4.8,877,499,0
4,29952,https://tiki.vn/cua-hang/hong-hanh-mobile,Hồng Hạnh Mobile,file 1,done,NaN,NaN,2019,4.7,2800,15108,100
...,...,...,...,...,...,...,...,...,...,...,...,...
937,265554,https://tiki.vn/cua-hang/gor-shop,Gor Shop,NaN,NaN,2024-03-08 00:00:00,NaN,2022,4.5,199,63,0
938,338,https://tiki.vn/cua-hang/nha-gia-kim,Nhà Giả Kim,NaN,NaN,2024-03-08 00:00:00,NaN,2017,4.5,2700,852,100
939,20411,https://tiki.vn/cua-hang/phu-kien-cho-ban-07,Phụ kiện cho bạn 07,NaN,NaN,2024-03-08 00:00:00,NaN,2019,4.5,6200,616,90
940,362,https://tiki.vn/cua-hang/cong-nghe-song-tin,Công nghệ Song Tín,NaN,NaN,2024-03-08 00:00:00,NaN,2017,3.8,928,138,0


# ***`Save`***

In [ ]:
df_combined_all.to_excel('/content/drive/MyDrive/Colab Notebooks/Dataset TIki/df_combined_all.xlsx')